# Search the Data

A very quick search utility for finding things on the map.

In [ ]:
import glob
import json
import os
import pandas as pd

## Load Data

In [ ]:
DATA_PATH = '../map/output/district/'

def list_district_data(path):
    return glob.glob(os.path.join(path, '*.data.json'))

def read_district_data(pathname):
    with open(pathname) as file:
        return json.load(file)

raw_data = [
    read_district_data(district_data_file)
    for district_data_file in list_district_data(DATA_PATH)
]

## Build Index

In [ ]:
from whoosh.index import create_in
from whoosh.fields import *
SCHEMA = Schema(
    my_eu_id=ID(stored=True),
    postcode=ID(stored=True),
    title=TEXT(stored=True),
    content=TEXT(stored=True))
INDEX = create_in("output", SCHEMA)

In [ ]:
def build_index(raw_district_data):
    writer = INDEX.writer()
    outward_code = raw_district_data['outwardCode']
    
    def make_postcode(row):
        return '{} {}'.format(outward_code, row['inwardCode'])
    
    def add_cordis(df):
        for _index, row in df[~df.objective.isna()].iterrows():
            writer.add_document(
                postcode=make_postcode(row),
                my_eu_id=row['myEuId'],
                title=row['projectTitle'],
                content=row['objective']
            )
                
    def add_creative_or_erasmus(df):
        for _index, row in df[~df.summary.isna()].iterrows():
            writer.add_document(
                postcode=make_postcode(row),
                my_eu_id=row['myEuId'],
                title=row['project'],
                content=row['summary']
            )

    def add_esif(df):
        for _index, row in df[~df.summary.isna()].iterrows():
            writer.add_document(
                postcode=make_postcode(row),
                my_eu_id=row['myEuId'],
                title=row['projectTitle'],
                content=row['summary']
            )
    
    for dataset, data in raw_district_data['datasets'].items():
        df = pd.read_json(json.dumps(data), orient='split')
        if dataset == 'cordis':
            add_cordis(df)
        if dataset == 'creative' or dataset == 'erasmus':
            add_creative_or_erasmus(df)
        if dataset == 'esif':
            add_esif(df)
            
    writer.commit()

for raw_district_data in raw_data:
    build_index(raw_district_data)

## Search

In [ ]:
from whoosh.query import *

def search(word, output_file_name=None, limit=20):
    if output_file_name is None:
        output_file_name = '{}.html'.format(word)
    query = Or([Term('title', word), Term('content', word)])
    with INDEX.searcher() as searcher:
        with open(output_file_name, 'w') as output_file:
            output_file.write('<html><body>')
            hits = searcher.search(query, limit=limit)
            hits.fragmenter.surround = 100
            output_file.write('<h1>{} hits for {}</h1>'.format(len(hits), word))
            output_file.write('<dl>')
            for hit in hits:
                output_file.write(
                    '<dt><a href="https://www.myeu.uk/#/postcode/{}">{}</a> ({})</dt>'.format(
                        hit['postcode'].replace(' ', '/'),
                        hit['postcode'],
                        hit['my_eu_id']
                    ))
                output_file.write('<dd><p style="text-decoration: underline;">')
                if hit.highlights('title') == '':
                    output_file.write(hit['title'])
                else:
                    output_file.write(hit.highlights('title'))
                output_file.write('</p><p>')
                output_file.write(hit.highlights('content', top=5))
                output_file.write('</p></dd>')
            output_file.write('</dl></body></html>')                

In [ ]:
search('women', limit=200)